# 단일 입력 PromptTemplate

In [4]:
from langchain.prompts import PromptTemplate

In [5]:
import langchain
langchain.__version__

'0.3.25'

In [6]:
template_str=(
    '당신은 최고 수준의 마케팅 카피라이터입니다. \n'
    '아래 제품의 매력적인 홍보 문구를 200자 내외로 작성하십시오. \n\n'
    '제품명: {product_name}'
)
prompt = PromptTemplate.from_template(template_str)
formatted_prompt= prompt.format(product_name='초슈퍼카메라 x100')
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터입니다. 
아래 제품의 매력적인 홍보 문구를 200자 내외로 작성하십시오. 

제품명: 초슈퍼카메라 x100


In [7]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4.1-nano',temperature=0)
chain=prompt|llm
result=chain.invoke({'product_name':formatted_prompt})
print(result.content)

초슈퍼카메라 x100으로 순간을 놓치지 마세요! 선명한 화질과 빠른 반응속도로 어떤 순간도 생생하게 담아내는 최고의 선택. 프로처럼 찍는 즐거움, 지금 바로 경험하세요!


# 다중 입력 PromptTemplate

In [8]:
multi_template_str=(
    '아래 뉴스 기사 제목과 키워드입니다\n'
    '이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요\n\n'
    '제목: {title}\n'
    '키워드: {keywords}\n'
)

summary_prompt=PromptTemplate(template=multi_template_str, input_variables=['title','keywords'])
sample_title='인공지능 기술과 발전과 미래'
sample_keywords= '머신러닝, 딥러닝, 산업 혁신'
formatted_summary_prompt=summary_prompt.format(title=sample_title, keywords=sample_keywords)
print(formatted_summary_prompt)

아래 뉴스 기사 제목과 키워드입니다
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요

제목: 인공지능 기술과 발전과 미래
키워드: 머신러닝, 딥러닝, 산업 혁신



In [9]:
result_summary=(summary_prompt | llm).invoke({
    'title': sample_title,
    'keywords': sample_keywords
})
print(result_summary.content)

인공지능 기술은 머신러닝과 딥러닝의 발전을 바탕으로 산업 전반에 혁신을 일으키고 있으며, 앞으로도 지속적인 발전을 통해 다양한 분야에서 새로운 가능성을 열어갈 것으로 기대됩니다.


# ChatPromptTemplate
- 역할 기반 프롬프트

In [10]:
from langchain.prompts import ChatPromptTemplate

In [11]:
# 시나리오 : 파이썬 프로그램 전무가 역할 부여, 사용자가 파이썬 관련 질문에 친절히, 간결한 답변 하도록
sys_message=('당신은 파이썬 분야의 뛰어난 전문가이자 조언자입니다.'
              '사용자의 프로그래밍 질문에 친절하고 이해하기 쉽게 답변하시오.'
              )
user_message='{question}'

chat_prompt = ChatPromptTemplate.from_messages([
    ('system', sys_message),
    ('user', user_message)
])
sample_question='파이썬의 리스트 컴프리헨션(list comprehension)과 map 함수의 차이가 무엇인가요?'
message=chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 파이썬 분야의 뛰어난 전문가이자 조언자입니다.사용자의 프로그래밍 질문에 친절하고 이해하기 쉽게 답변하시오.', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 리스트 컴프리헨션(list comprehension)과 map 함수의 차이가 무엇인가요?', additional_kwargs={}, response_metadata={})]


In [12]:
result=(chat_prompt | llm).invoke({'question',sample_question})
print(result.content)

안녕하세요! 파이썬의 리스트 컴프리헨션과 `map()` 함수는 둘 다 기존 시퀀스(리스트, 튜플 등)의 각 요소에 대해 어떤 작업을 수행해서 새 리스트를 만들 때 사용됩니다. 하지만 둘 사이에는 몇 가지 차이점이 있어요. 자세히 설명드릴게요.

### 1. 문법과 가독성
- **리스트 컴프리헨션**은 더 직관적이고 간결한 문법을 가지고 있어요.
  
  ```python
  # 예: 0부터 4까지의 제곱을 리스트로 만들기
  squares = [x**2 for x in range(5)]
  ```

- **`map()` 함수**는 함수와 시퀀스를 인자로 받아서, 그 함수가 시퀀스의 각 요소에 적용된 결과를 반환합니다.
  
  ```python
  def square(x):
      return x**2

  squares = list(map(square, range(5)))
  ```

### 2. 가독성과 사용 편의성
- 리스트 컴프리헨션은 한 줄로 간단하게 표현할 수 있어서 가독성이 좋아요.
- `map()`은 함수가 별도로 정의되어 있어야 하거나, 람다 함수를 사용할 수 있는데, 이 경우 가독성이 떨어질 수 있어요.

### 3. 함수와의 결합
- `map()`은 기존 함수를 재사용하거나, 이미 정의된 함수를 적용할 때 유용합니다.
- 리스트 컴프리헨션은 인라인으로 조건문이나 복잡한 표현식을 넣기 쉽습니다.

### 4. 조건문 사용
- 리스트 컴프리헨션은 조건문도 쉽게 넣을 수 있어요.
  
  ```python
  # 짝수만 제곱
  even_squares = [x**2 for x in range(10) if x % 2 == 0]
  ```

- `map()`은 조건문을 넣기 위해서는 별도의 함수 내에서 조건문을 써야 합니다.

### 5. 성능 차이
- 일반적으로 두 방법의 성능 차이는 크지 않지만, 간단한 작업에서는 리스트 컴프리헨션이 약간 더 빠를 수 있어요.

---

### 결론
- **리스트 컴프리헨션**은 간단하고 직관적이며, 조건문이

# PartialPromptTemplate

In [13]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [19]:
role_system_template='당신은 {role}분야의 전문 지식인입니다 가능한 한 자세히 답변해주세요'
system_prompt=SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template('{question}')

base_chat_prompt=ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt=base_chat_prompt.partial(role='주식 투자')

In [17]:
sample_question='현재 2025년 6월 시장 상황에서 삼성전자 주식 전망은 어때?'
message=partial_chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 주식 투자분야의 전문 지식인입니다 가능한 한 자세히 답변해주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 6월 시장 상황에서 삼성전자 주식 전망은 어때?', additional_kwargs={}, response_metadata={})]


In [20]:
result=(partial_chat_prompt | llm).invoke({'question',sample_question})
print(result.content)

2025년 6월 현재 시장 상황을 고려할 때, 삼성전자 주식의 전망은 여러 요인에 따라 달라질 수 있습니다. 아래에 주요 분석 포인트를 상세히 설명드리겠습니다.

1. 글로벌 반도체 시장 동향
- 삼성전자는 세계 최대 반도체 제조업체 중 하나로, D램과 낸드플래시 시장에서 강력한 위치를 차지하고 있습니다.
- 2024년 이후 글로벌 반도체 수요는 데이터 센터, AI, 5G, 자율주행차 등 첨단 기술 분야의 성장에 힘입어 지속적으로 증가하는 추세입니다.
- 그러나, 글로벌 공급망 문제, 원자재 가격 변동, 경쟁사(인텔, SK하이닉스 등)의 시장 점유율 경쟁이 지속되고 있어, 반도체 가격 변동성은 여전히 존재합니다.

2. 모바일 및 가전사업
- 삼성전자는 스마트폰(갤럭시 시리즈), 가전제품(TV, 냉장고, 세탁기 등) 부문에서도 강력한 브랜드 파워와 시장 점유율을 유지하고 있습니다.
- 2024년 이후 5G/6G 통신기술, 프리미엄 스마트폰 수요가 지속될 것으로 예상되며, 신기술 도입과 신제품 출시가 매출 성장에 긍정적 영향을 미칠 수 있습니다.

3. 기술 혁신 및 투자 전략
- 삼성전자는 인공지능, 파운드리(반도체 위탁생산), 차세대 디스플레이 등 미래 성장 동력에 대한 투자를 확대하고 있습니다.
- 특히, 파운드리 사업은 글로벌 고객사 확보와 첨단 공정 개발로 경쟁력을 강화하고 있어, 장기적 성장 기대가 높습니다.

4. 글로벌 경제 및 정책 환경
- 미국, 중국, 유럽 등 주요 시장의 경제 정책, 무역 정책, 규제 변화가 삼성전자 실적에 영향을 미칠 수 있습니다.
- 예를 들어, 미국의 반도체 지원 정책(반도체법)과 중국의 시장 규제 강화는 각각 기회와 도전 요인으로 작용할 수 있습니다.

5. 주가 전망 및 투자 전략
- 2025년 6월 현재, 삼성전자는 글로벌 경기 회복, 기술 혁신, 수요 증가 기대에 힘입어 긍정적인 전망을 갖고 있습니다.
- 그러나, 글로벌 경기 둔화, 경쟁 심화, 원자재 가격 변동성 등은 주가에 변동성을 유발할 수 있으니, 투자 